## Import Packages

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
%matplotlib inline

## Show images in grids

In [2]:
def show_images(images, cmap=None):
    cols = 2
    rows = (len(images)+1)//cols
    
    plt.figure(figsize=(10, 11))
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        # use gray scale color map if there is only one channel
        cmap = 'gray' if len(image.shape)==2 else cmap
        plt.imshow(image, cmap=cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

## Config input image dir

In [3]:
test_output_dir = 'test_images_output'
test_input_dir = 'test_images'
test_input_images = os.listdir(test_input_dir)

if test_output_dir not in os.listdir('./'):
    os.mkdir(test_output_dir)

## Read in an Image and plot

In [4]:
def show_all_test_image(processing_pipeline = []):
    images = []
    for image_url in test_input_images:
        image = mpimg.imread(os.path.join(test_input_dir, image_url))
        output = image
        for process in processing_pipeline:
            output = process(output)
        images.append(output)
    show_images(images)

## Apply mask
**Notes-** *Mainly yellow and color are used to build road strips*
1. Extract white strip by applying white mask
2. Extract yellow strip by applying yellow mask

In [5]:
def apply_mask(image):
    white_lower = np.uint8([200, 200, 200])
    white_upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(image, white_lower, white_upper)

    yellow_lower = np.uint8([140, 140,   0])
    yellow_upper = np.uint8([255, 255, 255])
    yellow_mask = cv2.inRange(image, yellow_lower, yellow_upper)
    
    graound_lower = np.uint8([130, 120, 110])
    graound_upper = np.uint8([255, 255, 255])
    graound_mask = cv2.inRange(image, graound_lower, graound_upper)
    
    color_mask = cv2.bitwise_or(white_mask, yellow_mask)

    graound_mask = cv2.bitwise_not(graound_mask)
    
    mask = cv2.bitwise_and(color_mask, graound_mask)
    
    white_image = cv2.bitwise_and(image, image, mask = white_mask)
    
    color_image = cv2.bitwise_or(image, image, mask = mask)
    
    return cv2.bitwise_or(color_image, white_image)

## Convert gray scale and gaussian blur

In [6]:
def smoothing_image(image, kernel_size=5):
    kernel = np.ones((kernel_size, kernel_size),np.int8)
    return cv2.filter2D(image, -1, kernel)
    
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def gaussian_blur(img, kernel_size = 5):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

## Select region of interest
**Select lane are region**

In [8]:
def region_of_interest(image):
    mask_one = np.zeros_like(image)       
    mask_two = np.zeros_like(image)       
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.1, rows]
    top_left     = [cols*0.46, rows*0.6]
    bottom_right = [cols, rows]
    top_right    = [cols*0.57, rows*0.6] 
    
    vertices_one = np.array(
        [
            [
                bottom_left,
                top_left, 
                [(top_right[0] + top_left[0])/2, top_right[1]], 
                [(bottom_right[0]+bottom_left[0])/2, bottom_right[1]]
            ]
        ], 
        dtype=np.int32
    )
    
    vertices_two = np.array(
        [
            [
                [(bottom_right[0]+bottom_left[0])/2, bottom_left[1]],
                [(top_right[0] + top_left[0])/2, top_left[1]], 
                top_right, 
                bottom_right
            ]
        ], 
        dtype=np.int32
    )
    
    
    cv2.fillPoly(mask_one, vertices_one, ignore_mask_color)
    cv2.fillPoly(mask_two, vertices_two, ignore_mask_color)
    
    masked_image_one = cv2.bitwise_and(image, mask_one)
    masked_image_two = cv2.bitwise_and(image, mask_two)

    return masked_image_one, masked_image_two

## Canny Edge detection

In [9]:
def canny_edge_detection(img, low_threshold = 80, high_threshold = 240):
    return cv2.Canny(img, low_threshold, high_threshold)

## Draw line function
**Functio draw line on road strip**

In [10]:
def add_zero_axis_point(lines, x, y):
    min_line = lines[0].reshape(4)
    max_line = lines[0].reshape(4)
    for i in range(1, len(lines)):
        x1, y1, x2, y2 = lines[i].reshape(4)
        m = max(min_line[1], min_line[3])
        ma = min(max_line[1], max_line[3])
        if(y1 > m or y2 > m):
            min_line = lines[i].reshape(4)
        if(y1 < m or y2 < m):
            max_line = lines[i].reshape(4)
    lines = lines.reshape((-1, 4)).tolist()
    if(min_line[1] < y or min_line[3] < y):
        minx,miny = [min_line[0], min_line[1]] if min_line[0] > min_line[2] else [min_line[2], min_line[3]]
        x1, y1, x2, y2 = min_line
        m = (y2-y1) / (x2-x1)
        b = miny - m * minx
        if m != 0:
            zero_x = int((y - b) / m)
            minL = [minx, miny, zero_x, y]
            lines.append(minL)
    if(min_line[1] < y or min_line[3] < y):
        maxx,maxy = [max_line[0], max_line[1]] if max_line[0] < max_line[2] else [max_line[2], max_line[3]]
        x1, y1, x2, y2 = max_line
        m = (y2-y1) / (x2-x1)
        b = miny - m * minx
        if m != 0:
            zero_x = int((y*0.61 - b) / m)
            maxL = [maxx, maxy, zero_x, int(y*0.61)]
            lines.append(maxL)
            
    return lines

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    
    if lines is not None:
        sorted_line = lines
    
        sorted_line.sort(axis = 1)
        
        avg_line = (sorted_line.sum(axis = 0) / len(sorted_line)).astype(int)
        
        y, x = img.shape[:2]
        zero_line = add_zero_axis_point(avg_line, x, y)  
        
        for line in zero_line:
            x1, y1, x2, y2 = line
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho = 2, theta = math.pi / 180, threshold = 40, min_line_len = 1, max_line_gap = 300):
    hough_lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, hough_lines)
    return line_img

## Add line to original image

In [11]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

## Build a Lane Finding Pipeline

Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [12]:
def process_test_image(image):
    color_mask = apply_mask(image)
    
    smooth_image = smoothing_image(color_mask, kernel_size=7)
    
    blur_gray = gaussian_blur(smooth_image, kernel_size=5)
    
    edges = canny_edge_detection(blur_gray)
    
    interest_region_one, interest_region_two = region_of_interest(edges)
    
    lines_left = hough_lines(interest_region_one)
    lines_right = hough_lines(interest_region_two)
    
            
    weighted_image_left = weighted_img(lines_left, image)
    weighted_image = weighted_img(lines_right, weighted_image_left)
    
    return weighted_image

for image_url in test_input_images:
    image = mpimg.imread(os.path.join(test_input_dir, image_url))
    output_image = process_test_image(image)
    mpimg.imsave(os.path.join(test_output_dir, image_url), output_image)

show_all_test_image(processing_pipeline=[process_test_image])

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [13]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [14]:
def process_image(image):
    color_mask = apply_mask(image)
    
    smooth_image = smoothing_image(color_mask, kernel_size=7)
    
    blur_gray = gaussian_blur(smooth_image, kernel_size=5)
    
    edges = canny_edge_detection(blur_gray)
    
    interest_region_one, interest_region_two = region_of_interest(edges)
    
    lines_left = hough_lines(interest_region_one)
    lines_right = hough_lines(interest_region_two)
    
    lines = cv2.bitwise_or(lines_left, lines_right)
            
    weighted_image = weighted_img(lines, image)
    
    return weighted_image

In [15]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:19<00:00, 11.20it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 9.33 s, sys: 293 ms, total: 9.62 s
Wall time: 21.4 s


In [21]:
HTML("""
<video width="960" heig\ht="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

In [17]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,7)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [01:05<00:00, 10.20it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 28.7 s, sys: 868 ms, total: 29.6 s
Wall time: 1min 6s


In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [19]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
## clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:44<00:00,  5.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 18.3 s, sys: 497 ms, total: 18.8 s
Wall time: 48.3 s


In [20]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))